In [3]:
cd ..

d:\OneDrive\GSD 5th term\Inverse Graphics\HW2\InverseGraphics-HW2\src


In [5]:
cd ..

d:\OneDrive\GSD 5th term\Inverse Graphics\HW2\InverseGraphics-HW2


In [6]:
import torch.nn.functional as F
from jaxtyping import Float
from omegaconf import DictConfig
from PIL import Image
from torch import Tensor
import torch
from src.dataset.field_dataset import FieldDataset
import numpy as np
from tests.f32 import f32

In [62]:
class FieldDatasetImage(FieldDataset):
    def __init__(self, cfg: DictConfig) -> None:
        """Load the image in cfg.path into memory here."""

        super().__init__(cfg)
        self.cfg = cfg
        read_image = Image.open(cfg.path).convert("RGB")

        self.image = (
            torch.tensor(np.array(read_image)).permute(2, 0, 1).unsqueeze(0).float()
            / 255
        )  # batch channel h w

    def query(
        self,
        coordinates: Float[Tensor, "batch d_coordinate"],
    ) -> Float[Tensor, "batch d_out"]:
        """Sample the image at the specified coordinates and return the corresponding
        colors. Remember that the coordinates will be in the range [0, 1].

        You may find the grid_sample function from torch.nn.functional helpful here.
        Pay special attention to grid_sample's expected input range for the grid
        parameter.
        """
        coordinates = coordinates * 2 - 1
        coordinates = coordinates.unsqueeze(0).unsqueeze(2)
        sampled_colors = F.grid_sample(self.image, coordinates)  # batch channel d_out 1
        return sampled_colors.squeeze(0).squeeze(-1).permute(1, 0)  # batch d_out

    @property
    def d_coordinate(self) -> int:
        return 2

    @property
    def d_out(self) -> int:
        return 3

    @property
    def grid_size(self) -> tuple[int, ...]:
        """Return a grid size that corresponds to the image's shape."""
        return self.image.shape[-2:]

In [63]:
def test_sampling():
    dataset = FieldDatasetImage(
        DictConfig(
            {
                "path": "data/tester.png",
            }
        )
    )

    coordinates = [
        [7 / 16, 7 / 16],
        [7 / 16, 9 / 16],
        [9 / 16, 7 / 16],
        [9 / 16, 9 / 16],
    ]

    expected = [
        [1, 0, 0],
        [0, 0, 1],
        [1, 1, 0],
        [0, 1, 0],
    ]

    assert torch.allclose(
        dataset.query(f32(coordinates)),
        f32(expected),
    )
    print("test successful")

In [64]:
test_sampling()

test successful


c:\Users\moumo\anaconda3\envs\ighw2\lib\site-packages\torch\nn\functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [11]:
def generate_samples(
        origins: Float[Tensor, "batch 3"],
        directions: Float[Tensor, "batch 3"],
        near: float,
        far: float,
        num_samples: int,
    ) -> tuple[
        Float[Tensor, "batch sample 3"],  # xyz sample locations
        Float[Tensor, "batch sample+1"],  # sample boundaries
    ]:
        """For each ray, equally divide the space between the specified near and far
        planes into num_samples segments. Return the segment boundaries (including the
        endpoints at the near and far planes). Also return sample locations, which fall
        at the midpoints of the segments.
        """
        num_rays = origins.shape[0]
        t_vals = torch.linspace(0.0, 1.0, steps=num_samples+1)
        z_vals = (far - near) * t_vals + near
        z_vals = z_vals.expand([num_rays, num_samples+1])
        mid_pts = 0.5 * (z_vals[..., 1:] + z_vals[..., :-1])
        print(f"mid_pts shape is {mid_pts.shape}")
        # stratified sampling
        
        np.random.seed(0)
        t_rand = torch.rand(*mid_pts.shape)
        upper = z_vals[..., 1:] - mid_pts
        lower = mid_pts - z_vals[..., :-1]
        z_vals_rand = mid_pts + upper * t_rand - lower * (1-t)*t_rand
        pts = (
            origins[..., None, :] + directions[..., None, :] * z_vals_rand[..., :, None]
        )  # [N_rays, N_samples, 3]
        boundaries = z_vals_rand

        return pts, boundaries

In [12]:
# Mock data for testing:
origins = torch.tensor([[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]])
directions = torch.tensor([[0.0, 0.0, 1.0], [0.0, 0.0, -1.0]])
near = 1.0
far = 10.0
num_samples = 4

# Call the function
pts, boundaries = generate_samples(origins, directions, near, far, num_samples)

# Print results
print("Sample Points (pts):")
print(pts.shape)
print("\nBoundaries:")
print(boundaries.shape)

mid_pts shape is torch.Size([2, 4])
Sample Points (pts):
torch.Size([2, 5, 3])

Boundaries:
torch.Size([2, 5])
